# preprocessing workflow 
for fMRI-data in the project 'Nothing compares to you'



## Literature
Peers example:
- segmentation in freeSurfer, coregister, realignment, artefact detection, smoothing, different masks using binaries
- no slice-time correction due to short TR, no normalization, no smoothing?

Casey et al. 2012
- AFNI: slice time correction, motion correction, despiking, artefact detection (head motion & linear trend)

Güclü et al.
- realignment to frist scan of the first run and next to the mean scan
- coregistration: anat to func mean
- slice-time correction
- normalization to mni

## Settings

In [1]:
# import modules
from os.path import join as opj
import os
from nipype.interfaces.freesurfer import (BBRegister, ApplyVolTransform,
                                          Binarize, RobustRegister, MRIConvert, FSCommand)
from nipype.interfaces.fsl import BET, BinaryMaths, ImageMaths, FLIRT, ConvertXFM
from nipype.interfaces.utility import Function, IdentityInterface, Merge
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.ants import ApplyTransforms
from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.spm import (Realign, Smooth)
import nipype.interfaces.spm as spm
from nipype.interfaces.ants import Registration
from nipype.interfaces.fsl import Info
from nipype.interfaces.c3 import C3dAffineTool

/home/mirjam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# SPM standalone
matlab_cmd = '/usr/local/spm12/run_spm12.sh /usr/local/MATLAB/MATLAB_Compiler_Runtime/v713 script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [3]:
# FreeSurfer - Specify the location of the freesurfer folder
fs_dir = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/derivatives/derivatives/mindboggle/freesurfer_subjects' # placeholder!
FSCommand.set_default_subjects_dir(fs_dir)

## specify parameters

In [9]:
experiment_dir = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/' # location of experiment folder
subject_list = ['sub-01'] # create the subject_list variable

output_dir = 'derivatives/preprocessing/output_preproc' # name of output folder
working_dir = 'derivatives/preprocessing/working_preproc' # name of working directory

number_of_slices = 25   # number of slices in volume
TR = 1.5    # time of repetition of volume
fwhm_size = 2 # smoothing kernel
iso_size= 2.5


## Import Data using SelectFiles

In [10]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
templates = {'func': '{subject_id}/func/{subject_id}_task-NC2U_run-*_bold.nii.gz', # epi images
             'funcwh': '{subject_id}/func/{subject_id}_task-NC2UWH_bold.nii.gz', # t2 whole brain images
             'anat': 'derivatives/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/brain.mgz', # anatomical images
             'aparc_aseg': 'derivatives/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/aparc+aseg.mgz',
             'brainmask': 'derivatives/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/brainmask.mgz',
             'ribbon': 'derivatives/derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/ribbon.mgz'}
            # use templates to create mask of temporal lobe
selectfiles = Node(SelectFiles(templates,
                                base_directory=experiment_dir),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")


# create and specify nodes
## realignment, artifact detection, unzipping, coregistration, smoothing

In [11]:
## realignment
# Realign - correct for motion (epi), we also need the here produced mean for corgestration
realignepi = Node(spm.Realign(register_to_mean=True),
                   name="realignepi")

# Realign - correct for motion (functional whole brain; to gain mean image for coregistration of slab)
realignwh = Node(spm.Realign(register_to_mean=True),
                   name="realignwh")

# BET - Skullstrip realigned functional images
BETepi = Node(BET(frac=0.3,
                    robust=True,
                    output_type='NIFTI_GZ'),
                name="BETepi")

# BET - Skullstrip realigned functional images
BETwh = Node(BET(frac=0.65,
                    robust=True,
                    output_type='NIFTI_GZ'),
                name="BETwh")

## Artifact Detection - determine which of the images in the functional series
# are outliers. This is based on deviation in intensity or movement.
art = Node(ArtifactDetect(norm_threshold=1,
                          zintensity_threshold=3,
                          mask_type='file',
                          parameter_source='SPM',
                          use_differences=[True, False]),
           name="art")

## Unzip-Nodes
# Gunzip - unzip the functinal image (epi)
gunzip = MapNode(Gunzip(), name="gunzip", iterfield=['in_file'])

# Gunzip - unzip the functinal image (t2 whole brain)
gunzipwh = MapNode(Gunzip(), name="gunzipwh", iterfield=['in_file'])

# mriconvert - transforms .mgz-files to nii.gz.-files
mriconvert = Node(MRIConvert(), name='mriconvert')

## Coregistration
# FLIRT - coregister epi slab to whole brain
coreg_epi = Node(FLIRT(dof=6,
                      cost='leastsq',
                      output_type='NIFTI_GZ'),
                name='coreg_epi')

# FLIRT - coregister whole brain to anatomical
coreg_wh = Node(FLIRT(dof=6,
                       cost='mutualinfo',
                       bins=256,
                       output_type='NIFTI_GZ'),
                 name="coreg_wh")

# Convert the coreg_epi transformation to ANTS ITK format
convert2itk_epi = Node(C3dAffineTool(fsl2ras=True,
                                 itk_transform=True),
                   name='convert2itk_epi')

# Convert the coreg_wh transformation to ANTS ITK format
convert2itk_wh = Node(C3dAffineTool(fsl2ras=True,
                                 itk_transform=True),
                   name='convert2itk_wh')

# Concatenate coreg_epi and coreg_wh transforms into a list
concat = Node(ConvertXFM(concat_xfm=True),
                    name='concat')

# Apply coregistration warp to functional images
applywarp = MapNode(FLIRT(interp='spline',
                       apply_isoxfm=iso_size,
                       output_type='NIFTI',
                       datatype ='short'),
                 name="applywarp", iterfield=['in_file'])

# Apply coregistration warp to mean file
applywarp_mean = Node(FLIRT(interp='spline',
                            apply_isoxfm=iso_size,
                            output_type='NIFTI_GZ'),
                 name="applywarp_mean")

# Smooth - to smooth the images with a given kernel
smooth = Node(Smooth(fwhm=fwhm_size),
              name="smooth")

## nodes for temporal lobe mask

In [ ]:
# Binarize node - creates a binary map of the temporal lobe
binarizeTemporalLobe = Node(Binarize(out_type='nii.gz',
                                       match = [1015, 2015, # middletemporal (aparc+aseg labels)
                                                1030, 2030, # superiortemporal (aparc+aseg labels)
                                                1001, 2001, # bankssts
                                                1034, 2034, # transversetemporal (Heschls Gyrus)
                                                1035, 2035], # insula
                                       binary_file='binarized_temporal_lobe.nii.gz'), # output file
                            name='binarizeTemporalLobe')

# MRIConvert - to unzip output files (because we have an nii.gz file as output above)
mriconvert_temporal_mask = Node(MRIConvert(out_type='nii'),
                                 name='mriconvert_temporal_mask')

# transform anatomical mask (binary) to functional space
applyVolTrans_temporal_mask = Node(ApplyVolTransform(reg_header=True,
                                                      interp='nearest'),
                                    name='applyVolTrans_temporal_mask')

# Dilate node - dilates the binary brain mask
dilate_temporal_mask = Node(Binarize(dilate=1,
                                      min=0.5),
                              name='dilate_temporal_mask')

# Binarize node - binarizes mask again after transformation and saves it as an .nii file
binarizeDilatedTemporalMask = Node(Binarize(min=0.1, binary_file='dilated_temporal_mask.nii'),
                                   name='binarizeDilatedTemporalMask')

# mean functional mask to adjust temporal lobe mask so no more non-gray-matter-voxel will be included
# meanfuncmask - create a whole brain mask from the mean functional based on FSL's robust BET 
meanfuncmask = Node(interface=BET(mask=True,
                                  no_output=False,
                                  frac=0.3,
                                  robust=True,
                                  output_type='NIFTI',
                                  out_file='meanfunc'),
                       name='meanfuncmask')

# combine temporal mask and meanfuncmask 
aparc_robust_BET_mask = MapNode(interface=ImageMaths(suffix='_ribbon_robust_BET',
                                               op_string='-mas', output_type='NIFTI', out_file='aparc_robust_BET.nii'),
                                               iterfield=['in_file'],
                                  name='aparc_robust_BET_mask')


#  function to plot realign parameters

In [ ]:
# plot realign parameters
def plot_realign_parameters(subject_id, realigned_files):

    # Import necessary modules
    from os.path import join as opj 
    import numpy
    import matplotlib.pyplot as plt
    import glob

    experiment_dir = '/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/' # path to your experiment directory
    working_dir = 'working_preproc' # path to the working directory
    # Load the estimated parameters
    list_realign_parameter_files=glob.glob(opj(experiment_dir, working_dir, "preproc_nc2u", "subject_id_"+subject_id, "realign", "*.txt"))
    
        
    plot_realign_parameters=[]
    for realign_file in list_realign_parameter_files:
    
        movement=numpy.loadtxt(realign_file)
    
        plt.figure(figsize=(10,8))
        
        plt.axhline(0, color='red')
        
        # Create the plots with matplotlib
        plt.subplot(211)
        plt.title('translation')
        plt.ylabel('in mm')
        plt.plot(movement[:,0], label='x')
        plt.plot(movement[:,1], label='y')
        plt.plot(movement[:,2], label='z')
        
        plt.legend(loc='best', fancybox=True)
        
        plt.axhline(0, color='red', alpha=0.5)
        
        plt.subplot(212)
        plt.title('rotation')
        plt.ylabel('in degrees')
        plt.plot(movement[:,3], label='pitch')
        plt.plot(movement[:,4], label='roll')
        plt.plot(movement[:,5], label='yaw')
        
        plt.legend(loc='best', fancybox=True)
        
        plt.axhline(0, color='red', alpha=0.5)

        plot_realign_parameters=plt.tight_layout()
        
        plt.savefig(opj(experiment_dir, working_dir, "preproc_nc2u" "subject_id_"+subject_id, "realign", realign_file+".png"), bbox_inches='tight')
        
    return plot_realign_parameters
    
# plot realing parameters - plot realignment parameters and save it later on
plot_realign_parameters = Node(Function(input_names=['subject_id', 'realigned_files'],
                               output_names=['plot_realign_parameters'],
                               function=plot_realign_parameters),
                      name='plot_realign_parameters')

# create and connect preprocessing workflow

In [ ]:
# Create a preprocessing workflow
preproc_nc2u = Workflow(name='preproc_nc2u')
preproc_nc2u.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow - aparc / ribbon mask 
preproc_nc2u.connect([(gunzip, realignepi, [('out_file', 'in_files')]), # send unzipped files to realignment node (epi)
                 (gunzipwh, realignwh, [('out_file', 'in_files')]), # send unzipped files to realignment node (t2 wh)
                 (realignepi, BETepi, [('mean_image', 'in_file')]), # skullstrip realigned epi
                 (realignwh, BETwh, [('mean_image', 'in_file')]), # skullstrip realigned t2 whole brain
                 # coregistration
                 (BETepi, coreg_epi, [('out_file', 'in_file')]), # coregister epi slab to t2 whole brain 
                 (BETwh, coreg_epi, [('out_file', 'reference')]), # coregister epi slab to t2 whole brain
                 (BETwh, coreg_wh, [('out_file', 'in_file')]), # coregister t2 whole brain to anatomical brain.mgz
                 (mriconvert, coreg_wh, [('out_file', 'reference')]),
                 # concat transform matrizes
                 (coreg_epi, concat, [('out_matrix_file', 'in_file')]),
                 (coreg_wh, concat, [('out_matrix_file', 'in_file2')]),
                 # apply coregistration
                 (concat, applywarp_mean, [('out_file', 'in_matrix_file')]),
                 (BETepi, applywarp_mean, [('out_file', 'in_file')]),
                 (mriconvert, applywarp_mean, [('out_file', 'reference')]),
                 (concat, applywarp, [('out_file', 'in_matrix_file')]),
                 (mriconvert, applywarp, [('out_file', 'reference')]),
                 # smoothing, artifact detection, plot realign parameters
                 (applywarp, smooth, [('out_file', 'in_files')]), # send applywarped files to smooth node
                 (realignepi, art, [('realigned_files', 'realigned_files')]),
                 (realignepi, art, [('mean_image', 'mask_file'),
                                 ('realignment_parameters', 'realignment_parameters')]),
                 (realignepi, plot_realign_parameters, [('realigned_files', 'realigned_files')]), # send realign files / parameters to plot realignment parameters node
                 # temporal mask     
                 (binarizeTemporalLobe, mriconvert_temporal_mask, [('binary_file', 'in_file')]), # convert ribbon file (temporal mask) to nifti
                 (mriconvert_temporal_mask, applyVolTrans_temporal_mask,[('out_file','source_file')]), # transform ribbon file (temporal mask) to functional space
                 (realignepi, applyVolTrans_temporal_mask, [('mean_image', 'target_file')]),
                 (applyVolTrans_temporal_mask, dilate_temporal_mask, [('transformed_file', 'in_file')]), # dilate transformed ribbon file (temporal mask)
                 (dilate_temporal_mask, binarizeDilatedTemporalMask, [('binary_file', 'in_file')]), # binarize dilated and transformed ribbon file (temporal mask)
                 # complete mask
                 (realignepi, meanfuncmask, [('mean_image', 'in_file')]), # create a skull stripped mask image from the mean functional
                 (meanfuncmask, aparc_robust_BET_mask, [('mask_file', 'in_file')]), # create a combined mask from ribbon file (cortical mask) and robust_BET mask
                 (binarizeDilatedTemporalMask, aparc_robust_BET_mask, [('binary_file', 'in_file2')])
                  ])

In [ ]:
# Connect infosource, selectfiles and datasink to the preprocessing workflow
preproc_nc2u.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                  (selectfiles, gunzip, [('func', 'in_file')]),
                  (selectfiles, applywarp, [('func', 'in_file')]),
                  (selectfiles, gunzipwh, [('funcwh', 'in_file')]),
                  (selectfiles, mriconvert, [('anat', 'in_file')]),
                  (selectfiles, binarizeTemporalLobe, [('aparc_aseg', 'in_file')]),
                  (realignepi, datasink, [('mean_image', 'realign.@mean'),
                                           ('realignment_parameters', 'realign.@parameters')]),
                       
                  (art, datasink,     [('outlier_files', 'art.@outliers'),
                                       ('plot_files', 'art.@plot')]),
                      
                  (coreg_epi, datasink, [('out_file','coreg_epi.@out_file'),
                                        ('out_matrix_file','coreg_epi.@out_matrix_file'),
                                        ('out_log', 'coreg_epi.@out_log')]),
                  (coreg_wh, datasink, [('out_file','coreg_wh.@out_file'),
                                         ('out_matrix_file','coreg_wh.@out_matrix_file'),
                                         ('out_log', 'coreg_wh.@out_log')]),
                  (applywarp, datasink, [('out_file','applywarp.@out_file'),
                                        ('out_matrix_file','applywarp.@out_matrix_file'),
                                        ('out_log', 'applywarp.@out_log')]),
                  (applywarp_mean, datasink, [('out_file','applywarp_mean.@out_file'),
                                        ('out_matrix_file','applywarp_mean.@out_matrix_file'),
                                        ('out_log', 'applywarp_mean.@out_log')]),
                  (mriconvert, datasink, [('out_file', 'mriconvert.@out_file')]),
                  (BETepi, datasink, [('out_file', 'BETepi.@out_file')]),
                  (BETwh, datasink, [('out_file', 'BETwh.@out_file')]),
                  (smooth, datasink, [('smoothed_files', 'smooth.@smoothed_files')]),
                  (binarizeDilatedTemporalMask, datasink,   [('binary_file', 'masks.@binarized_dilated_temporal_mask')]),
                  (aparc_robust_BET_mask, datasink, [('out_file', 'masks.@aparc_robust_BET_mask')]),
                  (infosource, plot_realign_parameters, [('subject_id','subject_id')]),
                  ])

# visualization

In [ ]:
%matplotlib inline
# Create a colored pipeline graph
preproc_nc2u.write_graph(graph2use='colored',format='png', simple_form=True)

In [ ]:
# Visualize the graph
from IPython.display import Image
Image('/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/derivatives/preprocessing/working_preproc/preproc_nc2u/graph.png')
#Image(filename=opj(preproc_nc2u.base_dir, 'preproc_nc2u', 'graph.dot.png'))

In [ ]:
# Create a detailed pipeline graph
preproc_nc2u.write_graph(graph2use='flat',format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image('/home/mirjam/Desktop/Masterarbeit/NC2U_BIDS/derivatives/preprocessing/working_preproc/preproc_nc2u/graph_detailed.png')

# run workflow

In [ ]:
#### run the workflow using multiple cores ####
preproc_nc2u.run('MultiProc', plugin_args={'n_procs':2})
#preproc_nc2u.run()